# Import And Functions


In [ ]:
import concurrent.futures
import os

import basedosdados as bd
import pandas as pd
from IPython.display import clear_output

In [ ]:
clear_output()

In [ ]:
def dicionatio_categoria(ano: int, categoria: str) -> list:
    dicionario = {
        "1": "Federal",
        "2": "Estadual",
        "3": "Municipal",
        "7": "Especial",
    }

    status = "Ativa" if ano == 2022 else "Inativa"

    if categoria in ("4", "5", "6", "8", "9"):
        return ["Privada", "Privada", status]
    else:
        return ["Público", dicionario[categoria], status]


def formar_linha(source_df: pd.DataFrame, id: str):
    mask = source_df.id_ies == id

    row = (
        source_df[mask]
        .sort_values(by="ano", ascending=False)
        .iloc[0]
        .to_list()
    )

    categoria = row.pop(3)
    row += dicionatio_categoria(ano=row[0], categoria=categoria)

    return row


def formar_df(source_df: pd.DataFrame) -> pd.DataFrame:
    columns = [
        "ano",
        "id_ies",
        "nome",
        "id_municipio",
        "sigla_uf",
        "tipo_instituicao",
        "rede",
        "situacao_funcionamento",
    ]

    ids_ies = source_df.id_ies.unique()

    with concurrent.futures.ThreadPoolExecutor() as executor:
        linhas = [
            row
            for row in executor.map(
                lambda id: formar_linha(source_df, id), ids_ies
            )
        ]

    df = pd.DataFrame(linhas, columns=columns)

    return df

# Gerar Tabela Atualizada


In [ ]:
billing_project_id = ""  # Definir o seu "billing_project_id" do google

query_educacao_superior = """
SELECT ano, id_ies, nome, tipo_categoria_administrativa, id_municipio, sigla_uf FROM `basedosdados.br_inep_censo_educacao_superior.ies`
"""
query_ensino_superior = """
SELECT * FROM `basedosdados.br_bd_diretorios_brasil.instituicao_ensino_superior`
"""
# Atual versão do "br_inep_censo_educacao_superior.ies" no BD
educacao_superior = bd.read_sql(
    query_educacao_superior, billing_project_id=billing_project_id
)
# Atual versão do "br_bd_diretorios_brasil.instituicao_ensino_superior" no BD
instituicao_ensino_superior = bd.read_sql(
    query_ensino_superior, billing_project_id=billing_project_id
)

df = formar_df(educacao_superior)

ensino_superior_to_merge = instituicao_ensino_superior[
    ~instituicao_ensino_superior.id_ies.isin(df.id_ies.values)
]

df_to_merge = df.loc[:, instituicao_ensino_superior.columns]

df_atualizada = pd.concat([ensino_superior_to_merge, df_to_merge])

df_atualizada.nome = df_atualizada.nome.str.title()

os.makedirs("output", exist_ok=True)

df_atualizada.to_csv("output/instituicao_ensino_superior.csv", index=False)